Copyright 2016 Google Inc. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

--------------------------------------

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
print(tf.__version__)
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
!python -V

In [ ]:
data_train_file = "../input/fashion-mnist_train.csv"
data_test_file = "../input/fashion-mnist_test.csv"

df_train = pd.read_csv(data_train_file)
df_test = pd.read_csv(data_test_file)


In [ ]:
df_train.head()

In [ ]:
# Select all columns but the first
features = df_train[df_train.columns[1:]]
features.head()

In [ ]:
labels = df_train['label']
labels.describe(include='all')

In [ ]:
features.values.shape

In [ ]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"pixels": features.values/255},
        y=labels,
        batch_size=100,
        num_epochs=3,
        shuffle=True)

In [ ]:
feature_columns = [tf.feature_column.numeric_column("pixels", shape=784)]

In [ ]:
classifier = tf.estimator.LinearClassifier(
                feature_columns=feature_columns, 
                n_classes=10,
                model_dir="./models/linear1"
                )

In [ ]:
classifier.train(input_fn=train_input_fn)


In [ ]:
features = df_test[df_test.columns[1:]]
labels = df_test["label"]

evaluate_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"pixels": features.values/255},
        y=labels,
        batch_size=100,
        num_epochs=1,
        shuffle=False)

In [ ]:
classifier.evaluate(input_fn=evaluate_input_fn)["accuracy"]

In [ ]:
# Bonus round 1: predictions
features = df_test[df_test.columns[1:]]
labels = df_test["label"]

predict_input_fn = tf.estimator.inputs.numpy_input_fn(        
        x={'pixels': features.iloc[5000:5005].values/255},
        batch_size=1,
        num_epochs=1,
        shuffle=False)
predictions = classifier.predict(input_fn=predict_input_fn)

for prediction in predictions:
    print("Predictions:    {} with probabilities {}\n".format(
        prediction["classes"], prediction["probabilities"]))
print('Expected answers values: \n{}'.format(
    labels.iloc[5000:5005]))

In [ ]:
# Bonus round 2: visualizing our predictions
# Import, setup, and a utility for int->string class conversion
import matplotlib.pyplot as plt
%matplotlib inline
class_table = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot"
]

def get_label_cls(label):
    """given an int label range [0,9], return the string description of that label"""
    return class_table[label]

get_label_cls(3)

In [ ]:
for i in range(5000,5005): 
    sample = np.reshape(df_test[df_test.columns[1:]].iloc[i].values/255, (28,28))
    plt.figure()
    plt.title("labeled class {}".format(get_label_cls(df_test["label"].iloc[i])))
    plt.imshow(sample, 'gray')

In [ ]:
DNN = tf.estimator.DNNClassifier(
                feature_columns=feature_columns, 
                hidden_units=[40,30,20],
                n_classes=10,
                model_dir="./models/deep1"
                )


In [ ]:
DNN.train(input_fn=train_input_fn)

In [ ]:
DNN.evaluate(input_fn=evaluate_input_fn)["accuracy"]